In [2]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/tests
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


In [18]:
from src.utils.embedding_utils import load_df_from_csv_with_embedding
from src.utils.config import OUTPUT_PATH

entities_with_embeddings=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_entities_with_embeddings_by_chatgpt4o.csv",['entity_embedding'])
# final_relationships=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_relationships_with_embeddings_by_chatgpt4o.csv",['relationship_embedding'])

In [2]:
import pandas as pd

entities_with_clusters=pd.read_csv("export/multihop_rag/filtered_entities_cluster_reults.csv")
entities_with_clusters

,entity_name,entity_type,description,entity_id,corpus_id,initial_leiden_cluster,merge_leiden_cluster,multihead_leiden_cluster_head_1,multihead_leiden_cluster_head_2,multihead_leiden_cluster_head_3
0,AMAZON,ORGANIZATION,Amazon is a retail giant known for its extensi...,0,0,1494,15,16,81,222
1,CYBER MONDAY,EVENT,Cyber Monday is a significant shopping event t...,1,0,1494,17,16,81,222
2,BLACK FRIDAY,EVENT,Black Friday is a shopping holiday that takes ...,2,0,1494,17,16,81,222
3,"NOVEMBER 27, 2023",TIME,"November 27, 2023, is the date when Cyber Mond...",3,0,1494,13,16,81,222
4,AMAZON,ORGANIZATION,Amazon is a multinational technology company k...,4,1,320,15,16,81,67
...,...,...,...,...,...,...,...,...,...,...
18083,PAUL POGBA,PERSON,Paul Pogba is a former football player for Uni...,18083,1999,1893,5,62,276,199
18084,BRUNO FERNANDES,PERSON,Bruno Fernandes is a football player with lead...,18084,1999,1893,5,62,276,199
18085,HARRY MAGUIRE,PERSON,Harry Maguire is a football player known for h...,18085,1999,1893,5,60,276,199
18086,CHELSEA,ORGANIZATION,Chelsea is a football club that competes with ...,18086,1999,3254,0,60,90,199


In [3]:
relationships=pd.read_csv("export/multihop_rag/filtered_relationships_with_duplicates.csv")
relationships

,source_entity,target_entity,relationship_description,relationship_strength,source_entity_id,target_entity_id,corpus_id
0,AMAZON,CYBER MONDAY,Amazon is hosting numerous deals during Cyber ...,9,0,1,0
1,AMAZON,BLACK FRIDAY,Amazon began its Black Friday sale a week ahea...,8,0,2,0
2,CYBER MONDAY,BLACK FRIDAY,"Cyber Monday follows Black Friday, with many d...",7,1,2,0
3,"NOVEMBER 27, 2023",CYBER MONDAY,"November 27, 2023, is the date that marks the ...",10,3,1,0
4,AMAZON,ECHO SHOW,"Amazon produces the Echo Show, integrating it ...",8,4,5,1
...,...,...,...,...,...,...,...
74170,ZIM,ZIM,SAME_NAME,8,5542,5551,609
74171,ZONE COVERAGE,ZONE COVERAGE,SAME_NAME,8,11185,11223,1229
74172,ZOOM,ZOOM,SAME_NAME,8,3168,3539,345
74173,ZOSHA MILLMAN,ZOSHA MILLMAN,SAME_NAME,8,12454,13789,1374


In [4]:
# 获取特定 entity_id 所在簇的节点数量
def get_cluster_node_count(df, entity_id, cluster_column):
    # 找到给定 entity_id 对应的簇标签
    cluster_label = df.loc[df['entity_id'] == entity_id, cluster_column].values[0]
    
    # 计算该簇中节点的数量
    cluster_node_count = df[df[cluster_column] == cluster_label].shape[0]
    
    return cluster_label, cluster_node_count

# 指定 entity_id
entity_id = 99  # 你要查找的 entity_id

# 聚类结果列名
cluster_columns = ['initial_leiden_cluster', 'merge_leiden_cluster'] + [col for col in entities_with_clusters.columns if col.startswith('multihead_leiden_cluster_head_')]

# 打印每个聚类所属簇的节点数量
for cluster_column in cluster_columns:
    cluster_label, node_count = get_cluster_node_count(entities_with_clusters, entity_id, cluster_column)
    print(f"Cluster label for {cluster_column} (entity_id {entity_id}): {cluster_label}")
    print(f"Number of nodes in the cluster: {node_count}\n")


Cluster label for initial_leiden_cluster (entity_id 99): 471
Number of nodes in the cluster: 8

Cluster label for merge_leiden_cluster (entity_id 99): 15
Number of nodes in the cluster: 293

Cluster label for multihead_leiden_cluster_head_1 (entity_id 99): 0
Number of nodes in the cluster: 458

Cluster label for multihead_leiden_cluster_head_2 (entity_id 99): 36
Number of nodes in the cluster: 184

Cluster label for multihead_leiden_cluster_head_3 (entity_id 99): 43
Number of nodes in the cluster: 176



In [5]:
from collections import deque
from src.gnn_clustering.utils import get_all_related_relationships_within_cluster

# 示例调用
entity_id = 0  # 你要查找的 entity_id
cluster_column = 'multihead_leiden_cluster_head_2'  # 聚类结果列
# initial_leiden_cluster	merge_leiden_cluster	multihead_leiden_cluster_head_1	multihead_leiden_cluster_head_2	multihead_leiden_cluster_head_3
related_relationships = get_all_related_relationships_within_cluster(entities_with_clusters, relationships, entity_id, cluster_column)

# 打印属于该 entity_id 的所有相关关系数量
print(len(related_relationships))
# 打印具体的关系
related_relationships


10


,source_entity,target_entity,relationship_description,relationship_strength,source_entity_id,target_entity_id,corpus_id
0,AMAZON,CYBER MONDAY,Amazon is hosting numerous deals during Cyber ...,9,0,1,0
1,AMAZON,BLACK FRIDAY,Amazon began its Black Friday sale a week ahea...,8,0,2,0
2,CYBER MONDAY,BLACK FRIDAY,"Cyber Monday follows Black Friday, with many d...",7,1,2,0
3,"NOVEMBER 27, 2023",CYBER MONDAY,"November 27, 2023, is the date that marks the ...",10,3,1,0
4,AMAZON,ECHO SHOW,"Amazon produces the Echo Show, integrating it ...",8,4,5,1
64,ANTITRUST LAWSUIT,AMAZON,The antitrust lawsuit is specifically targetin...,10,82,81,9
65,AMAZON,BUY BOX,Amazon's practices regarding the Buy Box impac...,7,81,83,9
66,AMAZON,PRICE DISCOUNTING STRATEGY,Amazon employs an anti-discounting strategy to...,8,81,84,9
67,AMAZON,WEB-CRAWLING APPARATUS,Amazon's web-crawling apparatus is used to tra...,8,81,85,9
68,AMAZON,PRICES,Amazon's pricing strategies have led to artifi...,9,81,86,9


In [14]:
from src.utils.message_templates import generate_community_report_template

message=generate_community_report_template(entity_id,entities_with_clusters[['entity_name', 'entity_type', 'description', 'entity_id', 'corpus_id']],related_relationships)

# 写入文件
with open('export/rag_test/community_report.txt', 'w') as file:
    for item in message:
        if 'content' in item:
            file.write(item['content'] + "\n\n")

In [15]:
from src.utils.llm_helpers import call_llm_api

response=call_llm_api(message)
# Save the message to a text file
with open('export/rag_test/response.txt', 'w') as file:
    file.write(response)

In [17]:
from collections import deque
from src.gnn_clustering.utils import get_all_related_relationships_within_cluster
from src.utils.message_templates import generate_community_report_template
import json

# 示例调用
entity_id = 999  # 你要查找的 entity_id
cluster_columns = [
    "initial_leiden_cluster", "merge_leiden_cluster", 
    "multihead_leiden_cluster_head_1", "multihead_leiden_cluster_head_2", 
    "multihead_leiden_cluster_head_3"
]

all_responses = []  # 存储所有 response
longest_response = None  # 记录 summary 最长的 response

# 遍历每个 cluster_column，生成消息并获取响应
for cluster_column in cluster_columns:
    related_relationships = get_all_related_relationships_within_cluster(
        entities_with_clusters, relationships, entity_id, cluster_column
    )
    
    message = generate_community_report_template(
        entity_id, 
        entities_with_clusters[['entity_name', 'entity_type', 'description', 'entity_id', 'corpus_id']],
        related_relationships
    )
    
    response = call_llm_api(message)  # 调用 LLM API 获取响应
    
    # 将字符串类型的 response 转换为字典
    response_dict = json.loads(response)
    all_responses.append(response_dict)

    # 检查当前 response 是否是 summary 最长的 response
    if not longest_response or len(response_dict["summary"]) > len(longest_response["summary"]):
        longest_response = response_dict

# 初始化合并后的结果
merged_response = {
    "title": longest_response["title"],
    "summary": longest_response["summary"],
    "findings": []
}

# 合并所有 findings
for response_dict in all_responses:
    merged_response["findings"].extend(response_dict["findings"])

# 将合并后的结果保存为 JSON 文件
with open('export/rag_test/merged_community_report.json', 'w') as file:
    json.dump(merged_response, file, ensure_ascii=False, indent=2)

# RAG检索

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# 假设你使用 Sentence-BERT 作为嵌入模型
model = SentenceTransformer('intfloat/multilingual-e5-large')

# 将问题转换为嵌入向量
def get_query_embedding(query):
    query_embedding = model.encode(query)
    return query_embedding

# 计算查询嵌入和实体嵌入之间的余弦相似度
def find_k_nearest_entities(query_embedding, final_entities, k=3):
    # 过滤掉嵌入为空的情况
    valid_entities = final_entities.dropna(subset=['entity_embedding'])
    # 确保所有嵌入向量的维度一致
    entity_embeddings = np.array([emb for emb in valid_entities['entity_embedding'].values if len(emb) == len(query_embedding)])
    similarities = cosine_similarity([query_embedding], entity_embeddings)[0]
    valid_entities = valid_entities.loc[valid_entities['entity_embedding'].apply(lambda x: len(x) == len(query_embedding))]
    valid_entities['similarity'] = similarities
    nearest_entities = valid_entities.sort_values(by='similarity', ascending=False).head(k)
    return nearest_entities

# 获取查询嵌入
query = "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"
query_embedding = get_query_embedding(query)

# 查询最相近的 k 个实体
k = 1
nearest_entities = find_k_nearest_entities(query_embedding, entities_with_embeddings, k)
nearest_entities

,entity_name,entity_type,description,entity_id,corpus_id,entity_embedding,similarity
13127,BANKMAN-FRIED,PERSON,Bankman-Fried is a key figure in the cryptocur...,13127,1452,"[0.01225276943296194, -0.012308556586503983, -...",0.857498


In [4]:
from src.report.community_report import generate_and_merge_reports

# entity_id=nearest_entities['entity_id'].iloc[0]
entity_id = 13127

merged_response=generate_and_merge_reports(entity_id, entities_with_clusters, relationships)

Merged report saved to export/rag_test/merged_community_report.json


In [13]:
import json
from src.utils.llm_helpers import call_llm_api

def generate_prompt(question, report):
    """
    根据问题和社区报告生成 Prompt，包含每个 finding 的 summary 和 explanation，
    并按照优化的结构进行组织。
    """
    findings = "\n".join([
        f"- {finding['id']}: {finding['summary']}\n  Explanation: {finding.get('explanation', 'No explanation provided.')}"
        for finding in report.get("findings", [])
    ])

    prompt = f"""
    ---Role---
    You are a helpful assistant responding to questions about data in the report provided.

    ---Goal---
    Generate a response to the following question based on the provided report:

    Question: {question}

    ---Instructions---
    - Use the data provided in the report below as the primary context for generating the response.
    - If you don't know the answer or if the input report does not contain sufficient information, respond with: "Information not found in the report."
    - Provide the `id` of the findings used to generate your response.
    - The response should be JSON formatted as follows:
      {{
          "answer": <string>,
          "used_findings": [<list of finding ids>]
      }}

    ---Context---
    Below is the community report data you have access to:

    Title: {report['title']}
    Summary: {report['summary']}

    Findings:
    {findings}

    output: 
    """

    return prompt

def call_llm_with_report(question, report):
    """
    生成 Prompt 并调用 LLM API 获取回答。
    """
    prompt = generate_prompt(question, report)
    message = [{"role": "user", "content": prompt}]
    
    print("Generated Prompt:\n", prompt)  # 检查生成的 Prompt
    
    # 调用 LLM API 获取响应
    response = call_llm_api(message)
    
    return response

# 示例调用
question = (
    "Who is the individual associated with the cryptocurrency industry facing a "
    "criminal trial on fraud and conspiracy charges, as reported by both The Verge "
    "and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"
)

# report = merged_response  # 加载生成的社区报告
def load_report(file_path):
    """
    加载 JSON 格式的社区报告。
    """
    with open(file_path, 'r') as file:
        return json.load(file)
report= load_report('export/rag_test/merged_community_report.json')
answer = call_llm_with_report(question, report)  # 调用 LLM 获取答案



Generated Prompt:
 
    ---Role---
    You are a helpful assistant responding to questions about data in the report provided.

    ---Goal---
    Generate a response to the following question based on the provided report:

    Question: Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?

    ---Instructions---
    - Use the data provided in the report below as the primary context for generating the response.
    - If you don't know the answer or if the input report does not contain sufficient information, respond with: "Information not found in the report."
    - Provide the `id` of the findings used to generate your response.
    - The response should be JSON formatted as follows:
      {
          "answer": <string>,
          "used_findings": [<list of finding ids>]
      }

    ---Context---
    Be

In [14]:
import json

def parse_llm_answer(answer):
    """
    解析 LLM 返回的 JSON 格式答案，并提取其中的内容。
    
    参数：
        answer (str): LLM 返回的 JSON 字符串。

    返回：
        dict: 包含解析后的答案和所用发现编号。
    """
    try:
        # 清理字符串，去除外部的 ```json ``` 包裹
        cleaned_answer = answer.strip("```").replace("json\n", "")
        
        # 将字符串转换为 Python 字典
        parsed_answer = json.loads(cleaned_answer)
        
        # 提取答案和所用发现
        answer_text = parsed_answer.get("answer", "No answer found.")
        used_findings = parsed_answer.get("used_findings", [])

        return {"answer": answer_text, "used_findings": used_findings}
    
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
        return None
    
parsed_result = parse_llm_answer(answer)

if parsed_result:
    print("Answer:", parsed_result["answer"])
    print("Used Findings:", parsed_result["used_findings"])

Answer: The individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges is Sam Bankman-Fried. He is accused by prosecutors of committing fraud for personal gain.
Used Findings: [1, 6, 12]
